In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [58]:
workspace_path = '/gdrive/My Drive/Colab Notebooks'

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os
import numpy as np
import matplotlib.pyplot as plt
# from keras.layers.convolutional import Conv2D, MaxPooling2D # 컨볼루션과 최대풀링을 위한 import

In [ ]:
class Net(nn.Module): #네트워크 구조개선(컨볼루션 신경망 사용)
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3,6,5) # (추가)입력채널을 세개, 출력채널을 6개로 설정하고 추출할 필터크기 5로 설정
        self.pool = nn.MaxPool2d(2,2) # (추가)2*2크기로 최대값 풀링
        self.conv2 = nn.Conv2d(6,32,5) # (추가)입력채널을 6개, 출력채널을 32개로 설정하고 추출할 필터크기 5로 설정

        self.fc1 = nn.Linear(32*5*5, 256)  # (수정) size에 맞게 800차원으로변경 후 256차원으로 축소
        self.fc2 = nn.Linear(256, 32) # 256차원에서 32차원으로 축소
        self.fc3 = nn.Linear(32, 10)  # 출력값의 차원은 판별할 클래스 수인 10으로 설정 (MNIST는 손글씨 숫자 10종 판별 문제)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x))) # 컨볼루션을 통과한 1번 레이어에 대한 풀링
        x = self.pool(F.relu(self.conv2(x))) # 컨볼루션을 통과한 2번 레이어에 대한 풀링
        #x = x.float()
        x = x.view(-1, 32*5*5 ) 
        h1 = F.relu(self.fc1(x))  # view 함수로 tensor 형태 변경 (배치수, axis_0, axis_1, ...)
        h2 = F.relu(self.fc2(h1))
        h3 = self.fc3(h2) 

        return h3

 
print("init model done")

In [ ]:
batch_size = 64  # 학습 배치 크기
test_batch_size = 1000  # 테스트 배치 크기 (학습 과정을 제외하므로 더 큰 배치 사용 가능)
max_epochs = 20  # (수정)학습 데이터셋 총 훈련 횟수 20회로 변경
lr = 0.001  # (수정)학습률 0.001로 변경
momentum = 0.95  # (수정)SGD에 사용할 모멘텀 설정 (0.95로 변경)
seed = 1  # 결과 재현을 위한 seed 설정
log_interval = 100  # (수정)interval 때마다 로그 남김  (100으로 변경)

use_cuda = torch.cuda.is_available()  # GPU cuda 사용 여부 확인

torch.manual_seed(seed)  # 결과 재현을 위한 seed 설정

device = torch.device("cuda" if use_cuda else "cpu")  # GPU cuda 사용하거나 없다면 CPU 사용

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  # num_workers: data loading할 프로세스 수, pin_memory: 고정된 메모리 영역 사용

print("set vars and device done")

In [62]:
transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])  # 임의의 값으로 초기화 (일반적으로는 학습 데이터셋의 평균, 표준편차 사용)

# CIFAR-10 link: https://www.cs.toronto.edu/~kriz/cifar.html
# 학습용 데이터 로더 (CIFAR-10 학습 데이터셋 사용)
train_loader = torch.utils.data.DataLoader(
  datasets.CIFAR10(os.path.join(workspace_path, 'data'), train=True, download=True, 
                   transform=transform), 
    batch_size = batch_size, shuffle=True, drop_last=True, **kwargs)  # drop_last: 마지막 미니배치 크기가 batch_size 이하면 drop 

# 테스트용 데이터 로더 (CIFAR-10 테스트 데이터셋 사용)
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10(os.path.join(workspace_path, 'data'), train=False, download=True,
                         transform=transform), 
    batch_size=test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


모델, 최적화 알고리즘, 손실 함수 정의

In [63]:
model = Net().to(device)  # 모델 정의
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # 최적화 알고리즘 정의 (SGD 사용)
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의 (CrossEntropy 사용)

AverageMeter 정의

In [64]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

학습, 테스트용 함수 정의

In [65]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()  # 모델 학습 모드 설정
    summary_loss = AverageMeter()  # 학습 손실값 기록 초기화
    summary_acc = AverageMeter() # 학습 정확도 기록 초기화
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 현재 미니 배치의 데이터, 정답 불러옴
        optimizer.zero_grad()  # gradient 0으로 초기화
        output = model(data)  # 모델에 입력값 feed-forward
        loss = criterion(output, target)  # 예측값(클래스 별 score)과 정답간의 손실값 계산
        loss.backward()  # 손실값 역전파 (각 계층에서 gradient 계산, pytorch는 autograd로 gradient 자동 계산)
        optimizer.step()  # 모델의 파라미터 업데이트 (gradient 이용하여 파라미터 업데이트)
        summary_loss.update(loss.detach().item())  # 손실값 기록
        pred = output.argmax(dim=1, keepdim=True)  # 예측값 중에서 최고 score를 달성한 클래스 선발
        correct = pred.eq(target.view_as(pred)).sum().item()  # 정답과 예측 클래스가 일치한 개수
        summary_acc.update(correct / data.size(0))  # 정확도 기록
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}, Accuracy: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), summary_loss.avg, summary_acc.avg))
            
    return summary_loss.avg, summary_acc.avg

def test(log_interval, model, device, test_loader):
    model.eval()  # 모델 검증 모드 설정 (inference mode)
    summary_loss = AverageMeter()  # 테스트 손실값 기록 초기화
    summary_acc = AverageMeter() # 테스트 정확도 기록 초기화
    with torch.no_grad():  # 검증 모드이므로 gradient 계산안함
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # 현재 미니 배치의 데이터, 정답 불러옴
            output = model(data)  # 모델에 입력값 feed-forward
            loss = criterion(output, target)  # 예측값(클래스 별 score)과 정답간의 손실값 계산
            summary_loss.update(loss.detach().item())  # 손실값 기록
            pred = output.argmax(dim=1, keepdim=True)  # 예측값 중에서 최고 score를 달성한 클래스 선발
            correct = pred.eq(target.view_as(pred)).sum().item()  # 정답과 예측 클래스가 일치한 개수
            summary_acc.update(correct / data.size(0))  # 정확도 기록

    print('\nTest set: Average loss: {:.4f}, Accuracy: {:.6f}\n'.format
          (summary_loss.avg, summary_acc.avg))  # 정답을 맞춘 개수 / 테스트셋 샘플 수 -> Accuracy

    return summary_loss.avg, summary_acc.avg

학습, 테스트, 모델 저장 수행

In [66]:
for epoch in range(1, max_epochs+1):
    best_acc = 0
    train_loss, train_acc = train(log_interval, model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(log_interval, model, device, test_loader)

    # 테스트에서 best accuracy 달성하면 모델 저장
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model, os.path.join(workspace_path, f'cifar10_ann_model_best_acc_{epoch}-epoch.pt'))
        print(f'# save model: cifar10_ann_model_best_acc_{epoch}-epoch.pt\n')

Train Epoch: 1 [0/50000 (0%)]	Average loss: 2.309775, Accuracy: 0.109375
Train Epoch: 1 [6400/50000 (13%)]	Average loss: 2.304802, Accuracy: 0.103496
Train Epoch: 1 [12800/50000 (26%)]	Average loss: 2.303153, Accuracy: 0.121035
Train Epoch: 1 [19200/50000 (38%)]	Average loss: 2.302257, Accuracy: 0.119913
Train Epoch: 1 [25600/50000 (51%)]	Average loss: 2.300597, Accuracy: 0.125896
Train Epoch: 1 [32000/50000 (64%)]	Average loss: 2.297907, Accuracy: 0.133171
Train Epoch: 1 [38400/50000 (77%)]	Average loss: 2.292181, Accuracy: 0.137297
Train Epoch: 1 [44800/50000 (90%)]	Average loss: 2.278976, Accuracy: 0.146264

Test set: Average loss: 2.0664, Accuracy: 0.247400

# save model: cifar10_ann_model_best_acc_1-epoch.pt

Train Epoch: 2 [0/50000 (0%)]	Average loss: 2.095066, Accuracy: 0.250000
Train Epoch: 2 [6400/50000 (13%)]	Average loss: 2.018207, Accuracy: 0.281250
Train Epoch: 2 [12800/50000 (26%)]	Average loss: 1.983736, Accuracy: 0.287547
Train Epoch: 2 [19200/50000 (38%)]	Average loss: